In [1]:
import geopandas as gpd
import folium
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

/Users/chenluyao/opt/anaconda3/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "2g")
    .config("spark.executer.memory", "4g")
    .getOrCreate()
)

In [4]:
sf = gpd.read_file("../data/external/SA2_2021/SA2_2021_AUST_GDA2020.shp")
sf = sf[["SA2_CODE21", "SA2_NAME21", "geometry"]]
sf.rename(columns={"SA2_CODE21":"SA2_code", "SA2_NAME21": "SA2_name"}, inplace=True)
sf = sf.loc[sf.geometry != None]
sf["SA2_code"] = sf["SA2_code"].astype(int)
gdf = gpd.GeoDataFrame(sf)

# create a JSON 
geoJSON = gdf.drop_duplicates("SA2_code").to_json()

# derive zone centroids 
gdf['centroid'] = gdf['geometry'].apply(lambda x: (x.centroid.y, x.centroid.x))
gdf.head()

,SA2_code,SA2_name,geometry,centroid
0,101021007,Braidwood,"POLYGON ((149.58424 -35.44426, 149.58444 -35.4...","(-35.45506362754262, 149.79323458580757)"
1,101021008,Karabar,"POLYGON ((149.21899 -35.36738, 149.21800 -35.3...","(-35.37594104823316, 149.23280174411747)"
2,101021009,Queanbeyan,"POLYGON ((149.21326 -35.34325, 149.21619 -35.3...","(-35.35102995469765, 149.22546319520652)"
3,101021010,Queanbeyan - East,"POLYGON ((149.24034 -35.34781, 149.24024 -35.3...","(-35.35516024135807, 149.25241255258334)"
4,101021012,Queanbeyan West - Jerrabomberra,"POLYGON ((149.19572 -35.36126, 149.19970 -35.3...","(-35.377580426117056, 149.20284846752082)"


In [5]:
sdf_all = spark.read.parquet("../data/curated/full_data_without_fraud/")
sdf_all.count()

13504142

In [6]:
def draw_map(df, columns, legend_name):
    """
    This function plots a choropleth map for a given dataframe and legend name.
    """
    map = folium.Map(location=[-30, 144], width=800, height=500, tiles="cartodbpositron", zoom_start=4)

    c = folium.Choropleth(
        geo_data=geoJSON, # geoJSON 
        name='choropleth', 
        data=df.reset_index(), # data source
        columns=columns, # the columns required
        key_on='properties.SA2_code', # this is from the geoJSON's properties
        fill_color='YlOrRd', # color scheme
        line_opacity=0.1,
        fill_opacity=0.7,
        legend_name=legend_name
    )
    c.add_to(map)
    return map

## Mean Total Income v.s. SA2 

In [7]:
# extract mean total income of each SA2 district
mean_income = sdf_all.select("SA2_code", "mean_total_income").distinct().dropna().toPandas()
mean_income["SA2_code"] = mean_income["SA2_code"].astype(int)
mean_income.head()

,SA2_code,mean_total_income
0,124041468,62728
1,405011113,50782
2,507011148,71203
3,215011387,60456
4,506061143,68793


In [8]:
# join the computed dataframe with geo dataframe
income_df = mean_income.merge(gdf[['SA2_code', 'SA2_name', 'geometry']], on='SA2_code')
income_df

,SA2_code,mean_total_income,SA2_name,geometry
0,124041468,62728,Yarramundi - Londonderry,"POLYGON ((150.67085 -33.64447, 150.66969 -33.6..."
1,405011113,50782,Mallala,"POLYGON ((138.25959 -34.44326, 138.25959 -34.4..."
2,507011148,71203,Banjup,"POLYGON ((115.86014 -32.14606, 115.86011 -32.1..."
3,215011387,60456,Ararat Surrounds,"POLYGON ((142.42681 -37.45834, 142.42710 -37.4..."
4,506061143,68793,Mundijong,"POLYGON ((115.88449 -32.23649, 115.88447 -32.2..."
...,...,...,...,...
1256,107011134,66663,Unanderra - Mount Kembla,"POLYGON ((150.79803 -34.42975, 150.79850 -34.4..."
1257,801091099,75035,Chifley,"POLYGON ((149.07195 -35.34808, 149.07309 -35.3..."
1258,316071545,54909,Bli Bli,"POLYGON ((152.97260 -26.60620, 152.97247 -26.6..."
1259,506011111,61102,Ashendon - Lesley,"POLYGON ((116.15935 -32.13593, 116.15930 -32.1..."


In [9]:
# find the two distrits that has very high mean income
income_df.loc[income_df["mean_total_income"]>200000]

,SA2_code,mean_total_income,SA2_name,geometry
1013,503011032,281099,Cottesloe,"POLYGON ((115.75101 -31.99292, 115.75101 -31.9..."
1153,503021037,276862,Kings Park (WA),"POLYGON ((115.83968 -31.96616, 115.83964 -31.9..."


In [10]:
# plot the map of mean total income for each SA2 district
income_map = draw_map(income_df, ['SA2_code','mean_total_income'], "Mean Total Income ($)")
income_map.save('../plots/mean_income_vs_SA2_map.html')

### Number of Consumers v.s. SA2

In [11]:
num_consumer = sdf_all.select("SA2_code", "consumer_id").distinct().dropna().groupBy("SA2_code").count().toPandas()
num_consumer["SA2_code"] = num_consumer["SA2_code"].astype(int)

In [12]:
# join the computed dataframe with geo dataframe
consumer_df = num_consumer.merge(gdf[['SA2_code', 'SA2_name', 'geometry']], on='SA2_code')
consumer_df.head()

,SA2_code,count,SA2_name,geometry
0,307011177,82,Roma Surrounds,"POLYGON ((147.44676 -26.11513, 147.45136 -26.0..."
1,115021298,18,Galston - Laughtondale,"MULTIPOLYGON (((151.15205 -33.52766, 151.15205..."
2,121031410,8,St Ives,"POLYGON ((151.15101 -33.72502, 151.14961 -33.7..."
3,211021261,18,Donvale - Park Orchards,"POLYGON ((145.17468 -37.78406, 145.17432 -37.7..."
4,510031273,32,Roebourne,"MULTIPOLYGON (((115.43450 -21.32744, 115.43378..."


In [13]:
# plot the map of mean total income for each SA2 district
num_consumer_map = draw_map(consumer_df, ['SA2_code','count'], "Number of Consumers")
num_consumer_map.save('../plots/number_of_consumer_vs_SA2_map.html')

### Number of Transactions v.s. SA2

In [14]:
num_transaction = sdf_all.select("SA2_code", "order_id").distinct().dropna().groupBy("SA2_code").count().toPandas()
num_transaction["SA2_code"] = num_transaction["SA2_code"].astype(int)

# join the computed dataframe with geo dataframe
transaction_df = num_transaction.merge(gdf[['SA2_code', 'SA2_name', 'geometry']], on='SA2_code')
transaction_df.head()

,SA2_code,count,SA2_name,geometry
0,115021298,10037,Galston - Laughtondale,"MULTIPOLYGON (((151.15205 -33.52766, 151.15205..."
1,121031410,4498,St Ives,"POLYGON ((151.15101 -33.72502, 151.14961 -33.7..."
2,208031189,5028,Edithvale - Aspendale,"POLYGON ((145.09815 -38.02479, 145.09805 -38.0..."
3,211021261,9994,Donvale - Park Orchards,"POLYGON ((145.17468 -37.78406, 145.17432 -37.7..."
4,307011177,46044,Roma Surrounds,"POLYGON ((147.44676 -26.11513, 147.45136 -26.0..."


In [15]:
# plot the map of mean total income for each SA2 district
num_transaction_map = draw_map(transaction_df, ['SA2_code','count'], "Number of Transactions")
num_transaction_map.save('../plots/number_of_transactions_vs_SA2_map.html')